# Train the Light-Head Mask-RCNN on the Coco Dataset
This notebook provides a fast way to start training on Coco dataset based on Light-Head Mask-RCNN.

In [1]:
"""
Based on the work of Waleed Abdulla (Matterport)
written by wozhouh
"""

# Import Python Packages
import os
import imgaug

# Root directory of the project
ROOT_DIR = os.path.abspath("../../../")
import sys

# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import model as modellib

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples", "coco"))  # To find local version
import coco

# # assign the GPU for training
# os.environ["CUDA_VISIBLE_DEVICES"] = "4"

Using TensorFlow backend.


In [2]:
# Default setting
HOME_DIR = os.getenv('HOME')
DEFAULT_WEIGHTS_DIR = os.path.join(ROOT_DIR, "weights")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
DEFAULT_DATASET_YEAR = "2017"
DEFAULT_COCO_PATH = os.path.join(HOME_DIR, "data", "Coco")

# weights to load
# MODEL_PATH_UNDER_HOME = os.path.join(DEFAULT_WEIGHTS_DIR, "ResNet-101", "mask_rcnn_coco.h5")
MODEL_PATH_UNDER_HOME = os.path.join(DEFAULT_LOGS_DIR, "Light-Head", "training20181211T1752", "mask_rcnn_training_0243.h5")
INIT_MODEL_PATH = os.path.join(HOME_DIR, MODEL_PATH_UNDER_HOME)
LOG_DIR = os.path.join(DEFAULT_LOGS_DIR, "Light-Head")

In [6]:
# training config
class TrainingConfig(coco.CocoConfig):
    NAME = "training"
    
    # GPU
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 2
    IMAGES_PER_GPU = 2
    
    # data
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # model
    BACKBONE = "resnet101"
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    
    # heads
    TOP_DOWN_PYRAMID_SIZE = 256
    DETECTION_HEAD = "light-head" 
    MASK_HEAD = "original"
    FPN_CLASSIF_FC_LAYERS_SIZE = 2048
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    LARGE_SEPARABLE_KERNEL_SIZE = 15
    LARGE_SEPARABLE_CHANNELS_MID = 256
    LARGE_SEPARABLE_CHANNELS_OUT = 490
    
    # training
    LEARNING_RATE = 0.001
    WEIGHT_DECAY = 0.0001
#     LOSS_WEIGHTS = {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
    LOSS_WEIGHTS = {'rpn_class_loss': 0.0, 'rpn_bbox_loss': 0.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.5, 'mrcnn_mask_loss': 0.0}
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 50
    TRAIN_BN = False
    TRAIN_ROIS_PER_IMAGE = 200
    ROI_POSITIVE_RATIO = 0.33
    
config = TrainingConfig()
config.display()
model = modellib.MaskRCNN(mode="training", model_dir=LOG_DIR, config=config)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_HEAD                 light-head
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     2048
GPU_COUNT                      2
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LARGE_SEPARABLE_CHANNELS_MID   256
LARGE_SEPARABLE_CHANNELS_OUT   490
LARGE_SEPARABLE_KERNEL_SIZE    15
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 0.0, 'r

In [4]:
# print the model summary
model.keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_image_meta (InputLayer)   (None, 93)           0                                            
__________________________________________________________________________________________________
input_rpn_match (InputLayer)    (None, None, 1)      0                                            
__________________________________________________________________________________________________
input_rpn_bbox (InputLayer)     (None, None, 4)      0                                            
__________________________________________________________________________________________________
input_gt_c

In [7]:
# Load the weights
model.load_weights(INIT_MODEL_PATH, by_name=True)

Re-starting from epoch 243


In [8]:
# Load the dataset
train_dataset = coco.CocoDataset()
train_dataset.load_coco(DEFAULT_COCO_PATH, "train", year=DEFAULT_DATASET_YEAR)
train_dataset.prepare()
val_dataset = coco.CocoDataset()
val_dataset.load_coco(DEFAULT_COCO_PATH, "val", year=DEFAULT_DATASET_YEAR)
val_dataset.prepare()

# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

loading annotations into memory...
Done (t=14.16s)
creating index...
index created!
loading annotations into memory...
Done (t=4.43s)
creating index...
index created!


In [9]:
# # # layers: "heads", "5+", "4+", "3+", "all", "light-head"
model.train(train_dataset=train_dataset,
                val_dataset=val_dataset,
                learning_rate=config.LEARNING_RATE / 20.0,
                epochs=391,
                layers='light-head-detection',
                augmentation=augmentation)


Starting at epoch 243. LR=5e-05

Checkpoint Path: /home/processyuan/code/HumanMask/my-Mobile-Mask-RCNN/logs/Light-Head/training20181211T1752/mask_rcnn_training_{epoch:04d}.h5
Selecting layers to train
In model:  rpn_model
In model:  large_separable_conv
    light_head_large_separable_conv_0a   (Conv2D)
    light_head_large_separable_conv_1a   (Conv2D)
    light_head_large_separable_conv_0b   (Conv2D)
    light_head_large_separable_conv_1b   (Conv2D)
    light_head_large_separable_bn   (BatchNorm)
light_head_class_conv   (TimeDistributed)
light_head_class_bn    (TimeDistributed)
light_head_bbox_fc     (TimeDistributed)
light_head_class_logits   (TimeDistributed)


/home/processyuan/anaconda3/envs/tflite/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/processyuan/anaconda3/envs/tflite/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 244/391
1000/1000 [==============================] - 1288s 1s/step - loss: 0.4901 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2052 - mrcnn_bbox_loss: 0.2849 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.7013 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2801 - val_mrcnn_bbox_loss: 0.4212 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 245/391
1000/1000 [==============================] - 1236s 1s/step - loss: 0.5437 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2279 - mrcnn_bbox_loss: 0.3157 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.7054 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2563 - val_mrcnn_bbox_loss: 0.4492 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 246/391
1000/1000 [==============================] - 1239s 1s/step - loss: 0.4868 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1983 - mrcnn_bbox_l

1000/1000 [==============================] - 1226s 1s/step - loss: 0.4806 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2081 - mrcnn_bbox_loss: 0.2725 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.5857 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.1727 - val_mrcnn_bbox_loss: 0.4130 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 265/391
1000/1000 [==============================] - 1222s 1s/step - loss: 0.4685 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1937 - mrcnn_bbox_loss: 0.2748 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.7064 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2537 - val_mrcnn_bbox_loss: 0.4527 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 266/391
1000/1000 [==============================] - 1245s 1s/step - loss: 0.5291 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2481 - mrcnn_bbox_loss: 0.2810 - 

1000/1000 [==============================] - 1280s 1s/step - loss: 0.4952 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2203 - mrcnn_bbox_loss: 0.2749 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.8070 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2913 - val_mrcnn_bbox_loss: 0.5157 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 285/391
1000/1000 [==============================] - 1289s 1s/step - loss: 0.5023 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2303 - mrcnn_bbox_loss: 0.2719 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.8016 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.3257 - val_mrcnn_bbox_loss: 0.4759 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 286/391
1000/1000 [==============================] - 1267s 1s/step - loss: 0.4427 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1956 - mrcnn_bbox_loss: 0.2471 - 

1000/1000 [==============================] - 1293s 1s/step - loss: 0.5650 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2715 - mrcnn_bbox_loss: 0.2935 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.6265 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2088 - val_mrcnn_bbox_loss: 0.4177 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 305/391
1000/1000 [==============================] - 1222s 1s/step - loss: 0.4311 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1882 - mrcnn_bbox_loss: 0.2430 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 1.0119 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.5100 - val_mrcnn_bbox_loss: 0.5018 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 306/391
1000/1000 [==============================] - 1206s 1s/step - loss: 0.4704 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1903 - mrcnn_bbox_loss: 0.2801 - 

1000/1000 [==============================] - 1222s 1s/step - loss: 0.4463 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1983 - mrcnn_bbox_loss: 0.2480 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 1.0608 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.4254 - val_mrcnn_bbox_loss: 0.6354 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 325/391
1000/1000 [==============================] - 1226s 1s/step - loss: 0.4991 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2284 - mrcnn_bbox_loss: 0.2708 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.6240 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2726 - val_mrcnn_bbox_loss: 0.3515 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 326/391
1000/1000 [==============================] - 1202s 1s/step - loss: 0.4343 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1815 - mrcnn_bbox_loss: 0.2528 - 

1000/1000 [==============================] - 1194s 1s/step - loss: 0.4362 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1951 - mrcnn_bbox_loss: 0.2411 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.6891 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2863 - val_mrcnn_bbox_loss: 0.4027 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 345/391
1000/1000 [==============================] - 1204s 1s/step - loss: 0.4944 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2239 - mrcnn_bbox_loss: 0.2704 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.7144 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2206 - val_mrcnn_bbox_loss: 0.4939 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 346/391
1000/1000 [==============================] - 1219s 1s/step - loss: 0.4319 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1855 - mrcnn_bbox_loss: 0.2464 - 

1000/1000 [==============================] - 1198s 1s/step - loss: 0.4476 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2014 - mrcnn_bbox_loss: 0.2462 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.6648 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.2815 - val_mrcnn_bbox_loss: 0.3833 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 365/391
1000/1000 [==============================] - 1204s 1s/step - loss: 0.4258 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1868 - mrcnn_bbox_loss: 0.2390 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.7747 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.3177 - val_mrcnn_bbox_loss: 0.4570 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 366/391
1000/1000 [==============================] - 1223s 1s/step - loss: 0.5075 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2211 - mrcnn_bbox_loss: 0.2864 - 

1000/1000 [==============================] - 1219s 1s/step - loss: 0.4691 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2146 - mrcnn_bbox_loss: 0.2544 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.7488 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.3143 - val_mrcnn_bbox_loss: 0.4346 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 385/391
1000/1000 [==============================] - 1203s 1s/step - loss: 0.4596 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.2035 - mrcnn_bbox_loss: 0.2560 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.7257 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.3134 - val_mrcnn_bbox_loss: 0.4123 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 386/391
1000/1000 [==============================] - 1194s 1s/step - loss: 0.4014 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1610 - mrcnn_bbox_loss: 0.2404 - 

In [ ]:
# Refer to the trained ResNet-101-based Mask-RCNN, loss should be 
# Epoch 1/30
#  28/1000 [..............................] - ETA: 1:37:39 
# - loss: 0.6337 
# - rpn_class_loss: 0.0112 
# - rpn_bbox_loss: 0.1044 
# - mrcnn_class_loss: 0.2009 
# - mrcnn_bbox_loss: 0.0904 
# - mrcnn_mask_loss: 0.2268